## Importing libraries

In [65]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np


## Extracting active and inactive for 'LLC','INC','LLP','Corp' and 'LP' and saving each as .csv file


In [67]:



keys = ['LLC','INC','LLP','Corp','LP']

active_id = "MainContent_rdbActive"
inactive_id = "MainContent_rdbInactive"


def get_all_active(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)

    search_type_dropdown = driver.find_element(By.ID, "MainContent_ddBeginsWithEntityName")
    select = Select(search_type_dropdown)
    select.select_by_visible_text("Full text")
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select1 = Select(dropdown_element)
    select1.select_by_visible_text("All items")
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'active'
    companies['entity_name'] = current
    companies.to_csv(f"companies_active_{current}.csv")
    print(f'All Actives for {current} ssuccessfully scraped')
    driver.quit()


def get_all_inactive(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')

    button = driver.find_element(By.ID, inactive_id)


    button.click()
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)
    
    search_type_dropdown = driver.find_element(By.ID, "MainContent_ddBeginsWithEntityName")
    select = Select(search_type_dropdown)
    select.select_by_visible_text("Full text")
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select1 = Select(dropdown_element)
    select1.select_by_visible_text("All items")
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'inactive'
    companies['entity_name'] = current
    companies.to_csv(f"companies_inactive_{current}.csv")
    print(f'All Inactives for {current} successfully scraped')
    driver.quit()
    companies

for i in keys:
    get_all_inactive(i)

for i in keys:
    get_all_active(i)


All Inactives for LLC successfully scraped
All Inactives for INC successfully scraped
All Inactives for LLP successfully scraped
All Inactives for Corp successfully scraped
All Inactives for LP successfully scraped
All Actives for LLC ssuccessfully scraped
All Actives for INC ssuccessfully scraped
All Actives for LLP ssuccessfully scraped
All Actives for Corp ssuccessfully scraped
All Actives for LP ssuccessfully scraped


## Merging the extracted .csv files

In [72]:
import pandas as pd
import glob
import os

csv_files = glob.glob(os.path.join(os.getcwd(), "*.csv"))

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.drop_duplicates(inplace=True)

#combined_df.to_csv("combined_output.csv", index=False)
combined_df


,Unnamed: 0,Entity Name,ID Number,NAICS,Inactive status,Address,status,entity_name
0,0,"CORP ONE REALTY, LLC",950754,531120,,"1 CORPORATE PLACE MIDDLETOWN, RI 02842 USA",active,Corp
1,1,Corpay,799539,522320,,"5301 MARYLAND WAY BRENTWOOD, TN 37027 USA",active,Corp
2,2,Corpay,137862,522390,,"1350 BROADWAY, SUITE 810 NEW YORK, NY 10018 USA",active,Corp
3,3,"Corpenna Development Group, LLC",148845,531190,,"23 CLARKES VILLAGE ROAD JAMESTOWN, RI 02835 USA",active,Corp
4,4,CORPO TRUCKING LLC,1732458,484121,Revoked Entity,"P.O. BOX 41463 PROVIDENCE, RI 02940 USA",active,Corp
...,...,...,...,...,...,...,...,...
413,70,"LPS Group, Inc.",130346,,Revoked Entity,", USA",inactive,LP
414,71,"LPS MARITIME, LLC",118715,487210,Revoked Entity,"3852 MAIN ROAD TIVERTON, RI 02878 USA",inactive,LP
415,72,LPS ON LINE ORDER LLC,516378,,Revoked Entity,"7 CONDUIT STREET LINCOLN, RI 02865 USA",inactive,LP
416,73,"LPT, LLC",485957,,Revoked Entity,"172 TAUNTON AVENUE, 1ST FLOOR EAST PROVIDENCE,...",inactive,LP


## Scraping per identification number page from "000000001" to "001775776"

### Date of incorporation not qualification

### The exact name: <span style="color: green;">✅</span>

### The fictitious name and filed date: <span style="color: red;">❌</span> Not Available

### Entity type: <span style="color: green;">✅</span>

### Identification Number: <span style="color: green;">✅</span>

### Date of Incorporation in Rhode Island: <span style="color: green;">✅</span>

### Effective Date: <span style="color: green;">✅</span>

### The location of the Principal Office: <span style="color: green;">✅</span>

### Agent Resigned: <span style="color: green;">✅</span>

### Address Maintained: <span style="color: green;">✅</span>

### The name and address of the Registered Agent: <span style="color: green;">✅</span>

### The Officers and Directors of the Corporation (if INC or Corp): <span style="color: green;">✅</span>

### The name and business address of each General Partner (if LP): <span style="color: green;">✅</span>

### The name and business address of each Manager (if LLC): <span style="color: green;">✅</span>

### The name and business address of each Partner who is authorized to execute, acknowledge, deliver, and record any recordable instrument purporting to affect any interest in real property: <span style="color: red;">❌</span> Not Available

### The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue: <span style="color: green;">✅</span>

### Purpose: <span style="color: green;">✅</span>

### North American Industry Classification System Code(NAICS): <span style="color: green;">✅</span>

### View filings for this business entity: <span style="color: green;">✅</span>


In [84]:

exact_names = []
entity_types = []
identification_numbers = []
date_of_incorporations = []
effective_dates = []
loc_principal_offices = []
agent_resigned = []
address_maintained = []
name_address_reg_agents = []
off_dir_corps = []
name_business_add_GPs = []
LLC_managers = []
stocks = []
purposes = []
NIACS = []
business_entities = []

# service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
# options = Options()
# ua = UserAgent()
# userAgent = ua.random
# options.add_argument(f'user-agent={userAgent}')

# driver = webdriver.Chrome(service=service,options=options)
# driver.get(f'https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN={id_num}&SEARCH_TYPE=3')

# time.sleep(2)
# page = driver.page_source


def get_more_details(id_num):
    
    url = f'https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN={id_num}&SEARCH_TYPE=3'
    print(id_num)
    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    page = requests.get(url, headers = headers)
    
    soup = BeautifulSoup(page.content, "html.parser")
    
    #exact_name
    if soup.find('span',attrs={'id': "MainContent_lblExactEntityName"}) is not None:
        exact_name = soup.find('span',attrs={'id': "MainContent_lblExactEntityName"}).text
        exact_names.append(exact_name)
    else:
        exact_names.append('None')
        
    #entity_type
    if soup.find('span',attrs={'id': "MainContent_lblEntityType"}) is not None:
        entity_type = soup.find('span',attrs={'id': "MainContent_lblEntityType"}).text
        entity_types.append(entity_type)
    else:
        entity_types.append('None')
    
    #identification number
    if soup.find('span',attrs={'id': "MainContent_lblIDNumber"}) is not None:
        identification_number = soup.find('span',attrs={'id': "MainContent_lblIDNumber"}).text.replace('Identification Number: ','')
        identification_numbers.append(identification_number)
    else:
        identification_numbers.append('None')

    #date of incorporation
    if soup.find('span',attrs={'id': "MainContent_lblOrganisationDate"}) is not None:
        date_of_incorporation = soup.find('span',attrs={'id': "MainContent_lblOrganisationDate"}).text
        date_of_incorporations.append(date_of_incorporation)
    else:
        date_of_incorporations.append('None')
    
    #effective date
    if soup.find('span',attrs={'id': "MainContent_lblEffectiveDate"}) is not None:
        effective_date = soup.find('span',attrs={'id': "MainContent_lblEffectiveDate"}).text
        effective_dates.append(effective_date)
    else:
        effective_dates.append('None')
    
    #location of principal office
    if soup.find('table',attrs={'id': "MainContent_tblPrincipleOffice"}) is not None:
        loc_principal_office = soup.find('table',attrs={'id': "MainContent_tblPrincipleOffice"}).text.replace('\xa0','').replace('\n','').replace('The location of the Principal Office:','').replace('Address:','').replace('City or Town, State, Zip, Country:',',')
        loc_principal_office = re.sub(r'\s+', ' ', loc_principal_office)
        loc_principal_offices.append(loc_principal_office)
    else:
        loc_principal_offices.append('None')


    #agent resigned
    if soup.find('span',attrs={'id': "MainContent_lblResidentAgentFlag"}) is not None:
        agent_resign = soup.find('span',attrs={'id': "MainContent_lblResidentAgentFlag"}).text
        agent_resigned.append(agent_resign)
    else:
        agent_resigned.append('None')
        
    
    #address maintained
    if soup.find('span',attrs={'id': "MainContent_lblConsentFlag"}) is not None:
        address_maintain = soup.find('span',attrs={'id': "MainContent_lblConsentFlag"}).text
        address_maintained.append(address_maintain)
    else:
        address_maintained.append('None')
    
    
    #The name and address of the Registered Agent
    if soup.find('table',attrs={'id': "MainContent_tblResident"}) is not None:
        name_address_reg_agent = soup.find('table',attrs={'id': "MainContent_tblResident"}).text.replace('Name:','').replace('\n','').replace('The name and address of the Registered Agent:','').replace('Address:',',').replace('City or Town, State, Zip, Country:',',')
        name_address_reg_agent = re.sub(r'\s+', ' ', name_address_reg_agent)
        name_address_reg_agents.append(name_address_reg_agent)
    else:
        name_address_reg_agents.append('None')
    
    
    #The Officers and Directors of the Corporation
    if soup.find('table',attrs={'id': "MainContent_tblOfficers"}) is not None:
        off_dir_corp = soup.find('table',attrs={'id': "MainContent_tblOfficers"}).text.replace('TitleIndividual NameAddress','').replace('\n','').replace('The Officers and Directors of the Corporation:','').replace('Address:',',').replace('City or Town, State, Zip, Country:',',')
        off_dir_corp = re.sub(r'\s+', ' ', off_dir_corp)
        off_dir_corps.append(off_dir_corp)
    else:
        off_dir_corps.append('None')
        
    #The name and business address of each General Partner
    if soup.find('table',attrs={'id': "MainContent_grdManagers"}) is not None:
        name_business_add_GP = soup.find('table',attrs={'id': "MainContent_grdManagers"}).text.replace('TitleIndividual nameAddress','').replace('\n','')
        name_business_add_GP = re.sub(r'\s+', ' ', name_business_add_GP)
        name_business_add_GPs.append(name_business_add_GP)
    else:
        name_business_add_GPs.append('None')



    #The limited liability company is to be managed by its Managers
    if soup.find('table',attrs={'id': "MainContent_tblManagers"}) is not None:
        LLC_manager = soup.find('table',attrs={'id': "MainContent_tblManagers"}).text.replace('The limited liability company is to be managed by its ManagersThe name and business address of each Manager:','').replace('\n','').replace('TitleIndividual nameAddress','').replace('The limited liability company is to be managed by its MembersThe name and business address of each Manager:','')
        LLC_manager = re.sub(r'\s+', ' ', LLC_manager)
        LLC_managers.append(LLC_manager)
    else:
        LLC_managers.append('None')
    
    
    
    #The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue
    if soup.find('table',attrs={'id': "MainContent_tblStocks"}) is not None:
        stock = soup.find('table',attrs={'id': "MainContent_tblStocks"}).text.replace('The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue:','').replace('\n','').replace('TitleIndividual nameAddress','')
        stock = re.sub(r'\s+', ' ', stock)
        stocks.append(stock)
    else:
        stocks.append('None')
    
    
    #Purpose
    if soup.find('table',attrs={'id': "MainContent_tblComments"}) is not None:
        purpose = soup.find('table',attrs={'id': "MainContent_tblComments"}).text.replace('Purpose:','').replace('\n','')
        purpose = re.sub(r'\s+', ' ', purpose)
        purposes.append(purpose)
    else:
        purposes.append('None')



    #North American Industry Classification System Code(NAICS)
    if soup.find('input',attrs={'id': "MainContent_txtNIACS","value": True}) is not None:
        NIAC = soup.find('input',attrs={'id': "MainContent_txtNIACS","value": True})
        NIAC = NIAC["value"]
        NIACS.append(NIAC)
    else:
        NIACS.append('None')
    
    
    
    #View filings for this business entity
    if soup.find('tr',attrs={'id': "MainContent_tdFilingList"}) is not None:
        business_entity = soup.find('tr',attrs={'id': "MainContent_tdFilingList"}).text.replace('\n',',').strip(',')
        business_entity = re.sub(r'\s+', ' ', business_entity)
        business_entities.append(business_entity)
    else:
        business_entities.append('None')


for i in combined_df['ID Number']:
    get_more_details(i)
    


extra_comp_data = {
    "Exact Names": exact_names,
    "Entity Types": entity_types,
    "Identification Numbers": identification_numbers,
    "Date of Incorporations": date_of_incorporations,
    "Effective Dates": effective_dates,
    "Location of Principal Offices": loc_principal_offices,
    "Agent Resigned": agent_resigned,
    "Address Maintained": address_maintained,
    "Name and Address of Registered Agents": name_address_reg_agents,
    "Officers, Directors, and Corporations": off_dir_corps,
    "Name and Business Address of General Partners": name_business_add_GPs,
    "LLC Managers": LLC_managers,
    "Stocks": stocks,
    "Purposes": purposes,
    "NIACS": NIACS,
    "Business Entities": business_entities
}

full_comp_df = pd.DataFrame(extra_comp_data)

full_comp_df

950754
799539
137862
148845
1732458
1751167
51147
1723774
579499
1705304
1680417
1701786
1745921
92809
960556
1686259
1732224
1724682
1724779
122561
44597
690404
1725395
498117
113518
113518
113518
60908
86828
1340878
54634
1695424
1666293
58568
58569
130159
104591
1716898
1735355
73863
123408
1743450
788906
1767222
1737505
84707
63422
1744709
55540
149198
30858
30860
22723
22723
22723
56859
1675384
875196
1763857
1660995
44913
1716898
1759068
1765554
1679889
1770400
1669390
1688184
1758898
1774032
142351
1722580
160122
1761568
66061
1720059
525840
1729590
1760611
1724283
306834
143095
157616
131604
111162
1657126
131259
147680
798972
1705496
49690
798972
152786
63098
89281
1753409
1727430
133023
1706494
1767793
1676467
1723944
91188
1753052
1734456
911807
793709
1761195
41152
23089
88427
1695773
68327
1772536
1008463
793089
1725468
157452
1677413
1049004
798630
1666023
1693715
1736944
1661684
152205
1762277
117105
149072
1712452
789551
88257
1753208
95400
799578
789443
1659319
115427


,Exact Names,Entity Types,Identification Numbers,Date of Incorporations,Effective Dates,Location of Principal Offices,Agent Resigned,Address Maintained,Name and Address of Registered Agents,"Officers, Directors, and Corporations",Name and Business Address of General Partners,LLC Managers,Stocks,Purposes,NIACS,Business Entities
0,The exact name of the Domestic Limited Liabili...,Domestic Limited Liability Company,000950754,06-20-2014,06-20-2014,"1 CORPORATE PLACE ,MIDDLETOWN, RI 02842 USA",N,Y,The name and address of the Resident Agent: PE...,None,,,None,"TO ACQUIRE, DEVELOP, OWN, HOLD FOR INVESTMENT...",531120 Lessors of Nonresidential Buildings (ex...,"ALL FILINGS,Annual Report,Annual Report - Amen..."
1,The exact name of the Foreign Corporation:,Foreign Corporation,000799539,06-05-2013,06-05-2013,"5301 MARYLAND WAY ,BRENTWOOD, TN 37027 USA",N,Y,"CORPORATION SERVICE COMPANY,222 JEFFERSON BOU...","PRESIDENT, SECRETARY, DIRECTORRANDY MORGAN 530...",None,None,Class of Stock Series Par value per share Tota...,"FINANCIAL TRANSACTIONS PROCESSING, RESERVE & ...","522320 Financial Transactions Processing, Rese...","ALL FILINGS,Annual Report,Annual Report - Amen..."
2,The exact name of the Foreign Corporation:,Foreign Corporation,000137862,02-10-2004,02-10-2004,"1350 BROADWAY, SUITE 810 ,NEW YORK, NY 10018 USA",N,Y,"CORPORATION SERVICE COMPANY,222 JEFFERSON BOU...","SECRETARYSHALOM WISE 212 KING STREET WEST, SUI...",None,None,Class of Stock Series Par value per share Tota...,PROVIDE FOREIGN CURRENCY EXCHANGE AND PAYMENT...,522390 Other Activities Related to Credit Inte...,"ALL FILINGS,Annual Report,Annual Report - Amen..."
3,The exact name of the Domestic Limited Liabili...,Domestic Limited Liability Company,000148845,06-21-2005,06-21-2005,"23 CLARKES VILLAGE ROAD ,JAMESTOWN, RI 02835 USA",N,Y,The name and address of the Resident Agent: PE...,None,,,None,REAL ESTATE DEVELOPMENT/CONTRACTOR TITLE: 7-16,531190 Lessors of Other Real Estate Property,"ALL FILINGS,Annual Report,Annual Report - Amen..."
4,The exact name of the Domestic Limited Liabili...,Domestic Limited Liability Company,001732458,11-24-2021,11-24-2021,"P.O. BOX 41463 ,PROVIDENCE, RI 02940 USA",N,N,The name and address of the Resident Agent: JO...,None,,,None,TRANSPORTING AMAZON LOADS,"484121 General Freight Trucking, Long-Distance...","ALL FILINGS,Annual Report,Annual Report - Amen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,The exact name of the Domestic Profit Corporat...,Domestic Profit Corporation,000130346,03-04-2003,03-04-2003,", USA",N,N,"LUDWIG P. SCHULZE,151 TABER AVENUE, #6 ,PROVI...","PRESIDENTLUDWIG P SCHULZE 151 TABER AVENUE, #6...",None,None,Class of Stock Series Par value per share Tota...,PROVIDING MANAGEMENT CONSULTING SERVICES TITL...,None,"ALL FILINGS,Annual Report,Annual Report - Amen..."
414,The exact name of the Domestic Limited Liabili...,Domestic Limited Liability Company,000118715,05-29-2001,05-29-2001,"3852 MAIN ROAD ,TIVERTON, RI 02878 USA",N,Y,The name and address of the Resident Agent: RI...,None,MANAGERANNMARIE SIROIS 113 PINELEDGE ROAD GREE...,The limited liability company is to be managed...,None,MARITIME TRADES TITLE: 7-16,"487210 Scenic and Sightseeing Transportation, ...","ALL FILINGS,Annual Report,Annual Report - Amen..."
415,The exact name of the Foreign Limited Liabilit...,Foreign Limited Liability Company,000516378,10-21-2009,10-21-2009,"7 CONDUIT STREET ,LINCOLN, RI 02865 USA",N,Y,The name and address of the Resident Agent: JI...,None,"MANAGERJIMMY ESTRADA 19 COBBLE HILL LINCOLN, R...",The limited liability company is to be managed...,None,INTERNET SALES OF COMPUTER PARTS,None,"ALL FILINGS,Amendment to Application for Regis..."
416,The exact name of the Domestic Limited Liabili...,Domestic Limited Liability Company,000485957,09-12-2008,09-12-2008,"172 TAUNTON AVENUE, 1ST FLOOR ,EAST PROVIDENCE...",N,N,The name and address of the Resident Agent: DA...,None,ManagerARTHUR B LEE 69 FREEMONT STREET PROVIDE..

In [83]:
NIACS

['531120 Lessors of Nonresidential Buildings (except Miniwarehouses) ',
 '522320 Financial Transactions Processing, Reserve, and Clearinghouse Activities ',
 '522390 Other Activities Related to Credit Intermediation ',
 '531190 Lessors of Other Real Estate Property ',
 '484121 General Freight Trucking, Long-Distance, Truckload ']

In [24]:
scrape the open data of the State of Rhode Island for all registered companies (3 different types)

Website: https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx

SEARCH CRITERIA:
Choose
"Active" and then "Inactive" after Scraping Active companies
1. "Search by entity name":
Search for:
LLC: Scrape all the companies
INC: Scrape all the companies
LLP: Scrape all the companies
Corp: Scrape all the companies
LP: Scrape all the companies

"Inactive" (after finish Scraping for Active companies), repeat the process with Inactive companies.
Note: There might be duplicate companies between the searches, you could delete the duplicates.

2. Or you can "Search by Identification Number":
Try from "000000001" to "001775776"

Fields:
-The exact name:

-The fictitious name and filed date (if have):

-Entity type:

-Identification Number:

-Date of Qualification in Rhode Island:

-Effective Date:

-The location of the Principal Office:

-Agent Resigned:

-Address Maintained:

-The name and address of the Registered Agent:

-The Officers and Directors of the Corporation (if INC or Corp):
1.Title:
2.Individual Name:
3.Address:

-The name and business address of each General Partner (if LP):
1.Title:
2.Individual Name:
3.Address:

-The limited liability company is to be managed by its Managers (if LLC)
-The name and business address of each Manager (if LLC):
1.Title:
2.Individual Name:
3.Address:

-The name and business address of each Partner who is authorized to execute, acknowledge, deliver, and record any recordable instrument purporting to affect any interest in real property (if LLP):
1.Title:
2.Individual Name:
3.Address:

-The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue (if have):

-Purpose:

-North American Industry Classification System Code(NAICS):

-View filings for this business entity:

Output:
Microsoft Excel file
Source Code (Python source code file must also be delivered to us)

SyntaxError: invalid decimal literal (257005525.py, line 44)

In [45]:
import re

# Sample list with excess white space in the middle of the string
address_list = ['205 AIRPORT ROAD ,WARWICK,                                                         RI                                                        02889-1029                                                        USA']

# Function to clean the string by removing excess white space
def clean_string(s):
    return re.sub(r'\s+', ' ', s)

# Apply the function to each string in the list
cleaned_list = [clean_string(s) for s in address_list]

# Print the cleaned list
print(cleaned_list)


['205 AIRPORT ROAD ,WARWICK, RI 02889-1029 USA']
